In [1]:
!python --version

Python 3.9.12


In [ ]:
!pip install pandas
!pip install konlpy
!pip install umap
!pip install -U scikit-learn
!pip install --upgrade pip
!pip install matplotlib
!pip3 install plotly

In [36]:
import os
os.getcwd()
import pandas as pd
path = "C:/Users/Notebiz003/BerTopic/data/"
file_list = os.listdir(path)
dfs = []

In [37]:
for i in file_list:
    df = pd.read_excel(path+i)
    dfs.append(df)
    print(i)

C:\Users\Notebiz003\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default



NewsResult_20180101-20181213.xlsx


C:\Users\Notebiz003\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default



NewsResult_20190101-20191231.xlsx


C:\Users\Notebiz003\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default



NewsResult_20200101-20201231.xlsx


C:\Users\Notebiz003\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default



NewsResult_20210101-20211231.xlsx


C:\Users\Notebiz003\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default



NewsResult_20220101-20220812.xlsx


In [38]:
from datetime import datetime


import logging
import re

import numpy as np
import pandas as pd

In [42]:
## 데이터 전처리
data = pd.concat(dfs)
data = data[['뉴스 식별자','일자','제목','키워드','특성추출(가중치순 상위 50개)']]
cluster_mode = '특성추출(가중치순 상위 50개)'
corpus = data[cluster_mode].values.tolist()

for i, cp in enumerate(corpus):
    corpus[i] = cp.split(',')
df = data[['일자']]
df['키워드'] = corpus
df['일자'] = [str(x)[0:6] for x in df['일자']] 


from_date = '2018-1-1'
to_date = '2022-7-31'
datelist = pd.date_range(start=from_date, end=to_date, freq = 'M').tolist()
dtlst = []

for d_t in datelist:
    d_t = str(d_t)[0:-9]
    d = datetime.strptime(d_t, '%Y-%m-%d')
    d = d.strftime('%Y%m')
    dtlst.append(d)
    
# 키워드 사전 
keyword_vocab = []
for i in df['키워드']:
    keyword_vocab.extend(i)
keyword_vocab = list(set(keyword_vocab))

# 월별 카운팅 후 병합
from collections import Counter

tmp_df_list = []
for dt in dtlst:
    tmp = []
    for i in df[df['일자']== dt]['키워드']:
        tmp.extend(i)
    tmp_df = pd.DataFrame(pd.Series(Counter(tmp)))
    tmp_df = tmp_df.reset_index()
    tmp_df = tmp_df.rename(columns = {0:'cnt','index':'keyword'})
    tmp_df_list.append(tmp_df)
    
df = tmp_df_list[0].merge(tmp_df_list[1], on = 'keyword', how = 'outer')
for i in range(len(tmp_df_list)):
    if i > 1:
        df = df.merge(tmp_df_list[i], on = 'keyword', how = 'outer')
col = ['keyword'] + dtlst
df.columns = col

# 합산 값 만들기
df.index = df['keyword']
df = df.drop('keyword', axis = 1)
df['tot_cnt'] = df.sum(axis = 1)
df = df.T
df['year_cnt'] = df.sum(axis = 1)
df = df.T
df

# 결측치 처리
df =df.fillna(0)
df

<ipython-input-42-0f7ba8fbab65>:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-42-0f7ba8fbab65>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-42-0f7ba8fbab65>:47: FutureWarning:

Passing 'suffixes' which cause duplicate columns {'cnt_x'} in the result is deprecated and will raise a MergeError in a future version.

<ipython-input-42-0f7ba8fbab65>:47: FutureWarning:

Passing 'suffixes' which cause duplicate columns {'cnt_x'} in the result is deprecated and will raise a Mer

,201801,201802,201803,201804,201805,201806,201807,201808,201809,201810,...,202111,202112,202201,202202,202203,202204,202205,202206,202207,tot_cnt
keyword,,,,,,,,,,,,,,,,,,,,,
협력사,2.0,0.0,4.0,0.0,1.0,4.0,0.0,0.0,9.0,9.0,...,2.0,0.0,7.0,10.0,6.0,8.0,3.0,1.0,5.0,206.0
결제대금,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
일본,52.0,17.0,39.0,28.0,14.0,16.0,23.0,26.0,26.0,34.0,...,29.0,42.0,38.0,13.0,16.0,37.0,29.0,36.0,37.0,1929.0
출연금,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,10.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,1.0,42.0
두산엔진,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
품질경영시스템,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
최홍훈,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
소비자보호시스템,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [6]:
# 데이터 확인
key = 'ai'
df[df.index == key]

,201801,201802,201803,201804,201805,201806,201807,201808,201809,201810,...,202111,202112,202201,202202,202203,202204,202205,202206,202207,tot_cnt
keyword,,,,,,,,,,,,,,,,,,,,,
ai,35.0,33.0,44.0,39.0,32.0,47.0,25.0,45.0,57.0,29.0,...,118.0,141.0,107.0,94.0,130.0,74.0,111.0,73.0,112.0,4453.0


In [7]:
# 데이터 비교1 . 회귀선의 기울기
# 연도별 건수 보정(normaliae)

df2 = df.copy()
for dt in dtlst:
    df2[dt] = df2[dt][:-1]*10000 / df2[dt][-1]
from scipy import stats

x = [x + 1for x in range(0,len(dtlst))]

slopes = []
for i in list(df2.values):
    y = i[:-1]
    slope, intercept, r, p, std_err = stats.linregress(x, y)
    slopes.append(slope)
df2['slope'] = slopes

In [18]:
df2.sort_values(by = 'slope',ascending = False)[0:5]

,201801,201802,201803,201804,201805,201806,201807,201808,201809,201810,...,202112,202201,202202,202203,202204,202205,202206,202207,tot_cnt,slope
keyword,,,,,,,,,,,,,,,,,,,,,
코로나19,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.0,...,49.700166,80.369140,56.579309,60.303182,44.852941,33.813785,28.679442,37.020061,7897.0,1.868031
코로나,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.0,...,18.900063,29.916588,28.682567,25.987006,21.323529,18.785436,11.297962,13.397736,3194.0,0.776734
메타버스,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.0,...,36.866790,45.889004,32.611685,37.314676,31.250000,20.493203,22.161387,29.263477,1411.0,0.695995
빅테크,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.89662,0.767401,0.0,...,28.233427,35.747788,19.645593,23.988006,34.191176,18.443883,21.292313,19.744033,2088.0,0.689092
가상자산,0.0,0.0,0.0,0.0,0.796813,0.0,0.0,0.00000,0.000000,0.0,...,21.233404,27.127754,22.003065,31.317674,26.102941,50.891454,40.411941,28.910905,1550.0,0.658254


In [17]:
df2.sort_values(by = 'slope',ascending = True)[0:5]

,201801,201802,201803,201804,201805,201806,201807,201808,201809,201810,...,202112,202201,202202,202203,202204,202205,202206,202207,tot_cnt,slope
keyword,,,,,,,,,,,,,,,,,,,,,
블록체인,69.167798,74.817208,76.869806,73.069540,60.557769,72.434608,50.568900,66.349861,59.089863,80.628905,...,19.600065,23.071267,23.181800,20.323172,26.102941,28.690484,21.726850,19.038889,3939.0,-0.921597
블록체,41.588233,62.064275,64.404432,61.923339,64.541833,74.446680,44.247788,64.556622,47.578850,77.941275,...,20.766736,20.535963,27.896743,31.650841,36.029412,26.299611,24.334072,30.673765,4192.0,-0.690907
빅데이터,23.201856,42.509777,67.174515,34.057836,50.996016,72.434608,55.625790,67.246481,69.833474,68.534570,...,26.133420,27.634815,12.180268,21.322672,20.220588,23.225630,16.077869,18.686317,4870.0,-0.587219
서울,64.790089,63.764666,59.556787,95.981175,77.290837,67.404427,75.853350,73.522819,64.461668,81.300813,...,54.833516,53.241386,44.791953,54.972514,56.985294,61.138056,65.615087,71.924691,7572.0,-0.463686
핀테크,31.081732,38.258800,58.864266,58.827172,44.621514,52.313883,46.144121,52.003945,51.415855,80.628905,...,35.233451,30.423649,26.325095,35.648842,48.161765,35.179999,43.019163,31.731481,6398.0,-0.461402


In [10]:
import plotly.io as pio
import plotly.express as px

In [19]:
srch_wrd = ['메타버스','가상자산','가상통화','블록체인','빅테크','핀테크','nft','esg','마이데이터','mz',
        '코로나19','가상화폐','공인인증서','오픈뱅킹','인공지능','디지털자산','암호화폐','공동인증서']

def Df3(srch_wrd):
    df3 = df.copy()
    df3 = df3.iloc[:,:-1]
    df3 = df3.loc[srch_wrd,:].reset_index()
    df3 = pd.melt(df3, id_vars='keyword', value_vars=list(df3))
    df3['variable'] = [x[:4] + '-' + x[4].replace('0','') + x[-1:] for x in df3['variable']]
    df3["variable"] = pd.to_datetime(df3.variable)
    df3.rename(columns={"variable": "Date"}, inplace=True)
    return df3

In [20]:
# 꺽은선 그래프
df3 = Df3(srch_wrd)
fig = px.line(
    df3, x="Date", y='value', color="keyword", title="Key Word (2018.1 - 2022.7)"
)
fig.show()

In [13]:
srch_wrd = ['가상자산','가상통화','가상화폐','암호화폐']
df3 =Df3(srch_wrd)
fig = px.bar(df3, x="Date", y="value", color="keyword", text="keyword")
fig.show()

In [16]:
df3 = Df[['공인인증서','공동인증서']]
df3['Percent'] = df3.groupby(['Date'])['value'].transform(lambda x: x/x.sum())
fig = px.bar(df3, x="Date", y="Percent", color="keyword", text="keyword")
fig.update_layout(bargap =0)
fig.show()

In [23]:
df3 = Df3(['공인인증서','공동인증서'])
df3['Percent'] = df3.groupby(['Date'])['value'].transform(lambda x: x/x.sum())
fig = px.bar(df3, x="Date", y="Percent", color="keyword", text="keyword")
fig.update_layout(bargap =0)
fig.show()

In [14]:

'''
# 시각화 비교 남는 시간 방법
x = []
y = []
keys = []
min_cnt = 300
for i in keyword_vocab:
    word = i
    if df['2021'][word] > min_cnt and df['2022'][word] > min_cnt:
        conf_2021 = df['2021'][word]/df['2021'][-1]
        conf_2022 = df['2022'][word]/df['2022'][-1]
        ratio_2022 = conf_2022 / conf_2021
        ratio_2021 = conf_2021 / conf_2022
        key = word
        x.append(ratio_2021)
        y.append(ratio_2022)
        keys.append(key)
df3 = pd.DataFrame({'key2021':x,'key2022':y}, index = keys)
df3 = df3.dropna()
import numpy as np
df3 = df3.dropna()
df3 = df3[df3.sum(axis = 1) != np.inf]
df2.sum(axis = 1)[0]
df3.sort_values(by = 'key2021',ascending = False).head(10)
df3.sort_values(by = 'key2022',ascending = False).head(10)
# 데이터 확인
key = '농협은행'
df[df.index == key]
'''